In [1]:
print("hello")

hello


In [2]:
import svgwrite

In [4]:
import cv2
import svgwrite
import numpy as np
import mediapipe as mp
from rembg import remove
from PIL import Image
import io
import matplotlib.pyplot as plt

# =========================================================
# [설정] 출력 및 드로잉 관련 상수
# =========================================================
NC_FILENAME = "face_drawing.nc"
SVG_FILENAME = "face_drawing.svg"

# G-code 생성 파라미터
Z_SAFE = 5.0      # 이동 시 높이 (펜 들기)
Z_DRAW = 0.0      # 그리기 높이 (펜 내리기)
FEED_RATE = 1000  # 이동 속도
SCALE = 0.3       # 이미지 좌표 -> G-code 좌표 스케일 (0.3배 축소)

# =========================================================
# [Block 1] 이미지 로드 및 배경 제거 (rembg)
# =========================================================
def load_and_remove_bg(path):
    """
    이미지 파일을 읽어 배경을 제거하고, 투명 영역을 흰색으로 채운 뒤 OpenCV 포맷으로 반환
    """
    print(f"🔄 [1단계] 배경 제거 시작: {path}")
    
    try:
        # 1. 파일 읽기 및 rembg로 배경 제거
        with open(path, 'rb') as i:
            input_data = i.read()
            output_data = remove(input_data)
        
        # 2. PIL 이미지로 변환
        img_pil = Image.open(io.BytesIO(output_data))
        
        # 3. 투명한 부분(Alpha)을 흰색으로 변경
        # (투명도가 있으면 나중에 검은색으로 인식될 수 있으므로 흰색 합성이 중요)
        background = Image.new("RGB", img_pil.size, (255, 255, 255))
        try:
            background.paste(img_pil, mask=img_pil.split()[3]) # Alpha 채널 사용
        except IndexError:
            background.paste(img_pil) # 알파 채널 없으면 그냥 붙임
            
        # 4. OpenCV 포맷(Numpy array)으로 변환 (RGB -> BGR)
        img_cv = np.array(background)
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
        
        print("✅ 배경 제거 완료")
        
        # (옵션) 주피터 노트북 등에서 확인용 - 필요 없으면 주석 처리
        # plt.imshow(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
        # plt.axis('off')
        # plt.show()
        
        return img_cv

    except Exception as e:
        print(f"❌ 이미지 로드 실패: {e}")
        return None

# =========================================================
# [Block 2] AI 얼굴 특징 추출 (MediaPipe)
# =========================================================
def extract_face_contours_mediapipe(img):
    """
    OpenCV 이미지를 받아 MediaPipe로 얼굴 주요 부위의 선(Contours)을 추출
    """
    print("🔄 [2단계] AI 얼굴 랜드마크 분석 중...")
    
    mp_face_mesh = mp.solutions.face_mesh
    height, width = img.shape[:2]
    
    # 얼굴 인식 설정
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,     # 정지 이미지 모드
        max_num_faces=1,            # 얼굴 1개만 인식
        refine_landmarks=True,      # 눈동자 등 정밀 인식
        min_detection_confidence=0.5
    ) as face_mesh:
        
        # MediaPipe는 RGB를 사용하므로 변환
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(img_rgb)
        
        if not results.multi_face_landmarks:
            print("⚠️ 얼굴을 찾지 못했습니다! (사람 정면 사진을 권장합니다)")
            return []

        landmarks = results.multi_face_landmarks[0].landmark
        
        # 추출할 얼굴 부위 정의 (MediaPipe Topology)
        # 딕셔너리에 각 부위의 연결선 정보를 담습니다.
        features = {
            "lips": mp.solutions.face_mesh.FACEMESH_LIPS,
            "left_eye": mp.solutions.face_mesh.FACEMESH_LEFT_EYE,
            "right_eye": mp.solutions.face_mesh.FACEMESH_RIGHT_EYE,
            "left_eyebrow": mp.solutions.face_mesh.FACEMESH_LEFT_EYEBROW,
            "right_eyebrow": mp.solutions.face_mesh.FACEMESH_RIGHT_EYEBROW,
            "face_oval": mp.solutions.face_mesh.FACEMESH_FACE_OVAL,
            "nose": mp.solutions.face_mesh.FACEMESH_NOSE
        }
        
        final_contours = []
        
        # 각 부위별 연결선을 순회하며 좌표 추출
        for name, connections in features.items():
            for connection in connections:
                start_idx = connection[0]
                end_idx = connection[1]
                
                pt1 = landmarks[start_idx]
                pt2 = landmarks[end_idx]
                
                # 정규화된 좌표(0~1)를 픽셀 좌표로 변환
                x1, y1 = int(pt1.x * width), int(pt1.y * height)
                x2, y2 = int(pt2.x * width), int(pt2.y * height)
                
                # 짧은 선분(Segment) 형태로 리스트에 추가
                # [[[x1, y1]], [[x2, y2]]] 형태는 cv2.drawContours와 호환됩니다.
                segment = np.array([[[x1, y1]], [[x2, y2]]], dtype=np.int32)
                final_contours.append(segment)

    print(f"✅ 얼굴 특징 추출 완료: 총 {len(final_contours)}개의 선분 생성")
    
    # (옵션) 디버깅용 이미지 저장
    debug_img = np.zeros((height, width), dtype=np.uint8)
    cv2.drawContours(debug_img, final_contours, -1, 255, 1)
    cv2.imwrite("debug_face_result.png", debug_img)
    print("   ㄴ 확인용 이미지 저장: debug_face_result.png")
    
    return final_contours

# =========================================================
# [Block 3] G-code 및 SVG 생성
# =========================================================
def save_gcode_and_svg(contours, width, height):
    """
    추출된 Contours를 G-code(.nc)와 SVG 파일로 저장
    """
    print(f"🔄 [3단계] 파일 생성 시작 ({NC_FILENAME}, {SVG_FILENAME})...")
    
    dwg = svgwrite.Drawing(SVG_FILENAME, profile='tiny', size=(width, height))
    
    count = 0
    with open(NC_FILENAME, 'w') as f:
        # G-code 헤더
        f.write("G21\nG90\nG0 Z10.0\n") # mm단위, 절대좌표, 안전높이 이동
        
        for contour in contours:
            # 점들의 배열로 변환 (Flatten)
            points = contour.reshape(-1, 2)
            
            if len(points) < 2: continue
            
            # --- SVG 저장 ---
            # numpy int32 -> python native float 변환 필요
            svg_data = [(float(p[0]), float(p[1])) for p in points]
            dwg.add(dwg.polyline(svg_data, stroke='black', fill='none', stroke_width=1))
            
            # --- G-code 저장 ---
            # 1. 시작점으로 이동 (펜 들고)
            start_x = float(points[0][0]) * SCALE
            start_y = float(points[0][1]) * SCALE
            f.write(f"G0 X{start_x:.3f} Y{-start_y:.3f}\n")
            
            # 2. 그리기 시작 (펜 내리고)
            f.write(f"G1 Z{Z_DRAW} F{FEED_RATE}\n")
            
            # 3. 경로 따라 이동
            for i in range(1, len(points)):
                px = float(points[i][0]) * SCALE
                py = float(points[i][1]) * SCALE
                f.write(f"G1 X{px:.3f} Y{-py:.3f}\n")
            
            # 4. 획 끝남 (펜 들기)
            f.write(f"G0 Z{Z_SAFE}\n")
            count += 1
            
        # G-code 푸터
        f.write("G0 X0 Y0\nM2\n")
        
    dwg.save()
    print(f"✅ 모든 작업 완료! 총 {count}개의 획이 저장되었습니다.")

# =========================================================
# [메인 실행부]
# =========================================================
if __name__ == "__main__":
    # 사용자 입력
    input_filename = input("파일 명을 입력하세요 (확장자 포함, 예: face.jpg) : ")
    
    # 1. 배경 제거 및 로드
    img_processed = load_and_remove_bg(input_filename)
    
    if img_processed is not None:
        # 2. AI로 얼굴 선 추출
        contours = extract_face_contours_mediapipe(img_processed)
        
        if contours:
            # 3. 파일 저장
            h, w = img_processed.shape[:2]
            save_gcode_and_svg(contours, w, h)

파일 명을 입력하세요 (확장자 포함, 예: face.jpg) :  input.png


🔄 [1단계] 배경 제거 시작: input.png
✅ 배경 제거 완료
🔄 [2단계] AI 얼굴 랜드마크 분석 중...
✅ 얼굴 특징 추출 완료: 총 149개의 선분 생성
   ㄴ 확인용 이미지 저장: debug_face_result.png
🔄 [3단계] 파일 생성 시작 (face_drawing.nc, face_drawing.svg)...
✅ 모든 작업 완료! 총 149개의 획이 저장되었습니다.


# 얼굴 안쪽 (MP-facemesh), 그외 (Canny)

In [5]:
import cv2
import svgwrite
import numpy as np
import mediapipe as mp
from rembg import remove
from PIL import Image
import io
import matplotlib.pyplot as plt

# =========================================================
# [설정] 파라미터
# =========================================================
NC_FILENAME = "face_drawing_detailed.nc"
SVG_FILENAME = "face_drawing_detailed.svg"
Z_SAFE = 5.0
Z_DRAW = 0.0
FEED_RATE = 1000
SCALE = 0.3

# =========================================================
# [Block 1] 이미지 로드 및 배경 제거 (기존 동일)
# =========================================================
def load_and_remove_bg(path):
    print(f"🔄 [1단계] 배경 제거 시작: {path}")
    try:
        with open(path, 'rb') as i:
            input_data = i.read()
            output_data = remove(input_data)
        
        img_pil = Image.open(io.BytesIO(output_data))
        background = Image.new("RGB", img_pil.size, (255, 255, 255))
        try:
            background.paste(img_pil, mask=img_pil.split()[3])
        except IndexError:
            background.paste(img_pil)
            
        img_cv = np.array(background)
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)
        print("✅ 배경 제거 완료")
        return img_cv
    except Exception as e:
        print(f"❌ 이미지 로드 실패: {e}")
        return None

# =========================================================
# [Block 2] 하이브리드 특징 추출 (핵심 변경!)
# =========================================================
def extract_hybrid_contours(img):
    print("🔄 [2단계] 하이브리드 처리 중 (AI 얼굴 + Canny 디테일)...")
    
    height, width = img.shape[:2]
    contours_list = []
    
    # -----------------------------------------------------
    # 1. MediaPipe로 얼굴 랜드마크 & 마스크 영역 구하기
    # -----------------------------------------------------
    mp_face_mesh = mp.solutions.face_mesh
    face_mask = np.zeros((height, width), dtype=np.uint8) # 얼굴 영역 마스크
    
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True, # 눈동자 인식 활성화
        min_detection_confidence=0.5
    ) as face_mesh:
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0].landmark
            
            # (A) 얼굴 외곽선 좌표 구하기 (마스킹용)
            face_oval_idx = mp_face_mesh.FACEMESH_FACE_OVAL
            oval_points = []
            for connection in face_oval_idx:
                pt = landmarks[connection[0]]
                x, y = int(pt.x * width), int(pt.y * height)
                oval_points.append((x, y))
            
            # 얼굴 영역을 흰색으로 칠해둠 (나중에 Canny에서 이 부분만 지울 거임)
            if oval_points:
                cv2.fillPoly(face_mask, [np.array(oval_points)], 255)
                # 마스크를 조금 축소 (Erode)해서 경계선이 자연스럽게 겹치게 함
                kernel = np.ones((15, 15), np.uint8)
                face_mask = cv2.erode(face_mask, kernel, iterations=1)

            # (B) AI로 예쁜 얼굴 선 추출 (눈, 코, 입, 눈동자)
            # 눈동자(Iris) 인덱스 추가 (MediaPipe 468~477번)
            # 리스트로 직접 정의하여 연결
            left_iris_indices = [474, 475, 476, 477, 474] # 닫힌 원
            right_iris_indices = [469, 470, 471, 472, 469] 
            
            # 주요 부위 정의
            features = {
                "lips": mp_face_mesh.FACEMESH_LIPS,
                "left_eye": mp_face_mesh.FACEMESH_LEFT_EYE,
                "right_eye": mp_face_mesh.FACEMESH_RIGHT_EYE,
                "left_eyebrow": mp_face_mesh.FACEMESH_LEFT_EYEBROW,
                "right_eyebrow": mp_face_mesh.FACEMESH_RIGHT_EYEBROW,
                "nose": mp_face_mesh.FACEMESH_NOSE,
                "face_oval": mp_face_mesh.FACEMESH_FACE_OVAL
            }

            # 1) 일반 부위 추가
            for name, connections in features.items():
                for connection in connections:
                    pt1, pt2 = landmarks[connection[0]], landmarks[connection[1]]
                    x1, y1 = int(pt1.x * width), int(pt1.y * height)
                    x2, y2 = int(pt2.x * width), int(pt2.y * height)
                    contours_list.append(np.array([[[x1, y1]], [[x2, y2]]], dtype=np.int32))
            
            # 2) 눈동자 추가 (수동 연결)
            def add_iris(indices):
                for i in range(len(indices) - 1):
                    pt1, pt2 = landmarks[indices[i]], landmarks[indices[i+1]]
                    x1, y1 = int(pt1.x * width), int(pt1.y * height)
                    x2, y2 = int(pt2.x * width), int(pt2.y * height)
                    contours_list.append(np.array([[[x1, y1]], [[x2, y2]]], dtype=np.int32))
            
            add_iris(left_iris_indices)
            add_iris(right_iris_indices)

    # -----------------------------------------------------
    # 2. Canny Edge로 나머지 디테일(머리, 옷) 구하기
    # -----------------------------------------------------
    # 흑백 변환 및 블러
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 엣지 검출 (임계값 조절로 디테일 결정: 낮을수록 자세함)
    canny_edges = cv2.Canny(blurred, 30, 100)
    
    # [핵심] 얼굴 마스크 반전 (얼굴이 아닌 부분만 남김)
    mask_inv = cv2.bitwise_not(face_mask)
    
    # 얼굴 부분의 Canny Edge 지우기 (AI가 그린 것과 겹치지 않게)
    canny_no_face = cv2.bitwise_and(canny_edges, canny_edges, mask=mask_inv)
    
    # -----------------------------------------------------
    # 3. Canny 결과를 Contour로 변환하여 리스트에 합치기
    # -----------------------------------------------------
    # 끊어진 선 연결 (Morphology)
    kernel_connect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    canny_dilated = cv2.dilate(canny_no_face, kernel_connect, iterations=1)
    
    # 외곽선 추출
    new_contours, _ = cv2.findContours(canny_dilated, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in new_contours:
        if cv2.arcLength(cnt, False) > 30: # 너무 짧은 잡음 제거
            # 점 개수 줄이기 (G-code 최적화)
            epsilon = 0.002 * cv2.arcLength(cnt, False)
            approx = cv2.approxPolyDP(cnt, epsilon, False)
            contours_list.append(approx)

    print(f"✅ 추출 완료: 총 {len(contours_list)}개의 획 (AI + 디테일)")
    
    # 디버깅 이미지 저장
    debug_img = np.zeros((height, width), dtype=np.uint8)
    cv2.drawContours(debug_img, contours_list, -1, 255, 1)
    cv2.imwrite("debug_hybrid_result.png", debug_img)
    
    return contours_list

# =========================================================
# [Block 3] G-code 및 SVG 생성 (기존 동일)
# =========================================================
def save_gcode_and_svg(contours, width, height):
    print(f"🔄 [3단계] 파일 생성 시작 ({NC_FILENAME})...")
    dwg = svgwrite.Drawing(SVG_FILENAME, profile='tiny', size=(width, height))
    
    count = 0
    with open(NC_FILENAME, 'w') as f:
        f.write("G21\nG90\nG0 Z10.0\n")
        
        for contour in contours:
            points = contour.reshape(-1, 2)
            if len(points) < 2: continue
            
            svg_data = [(float(p[0]), float(p[1])) for p in points]
            dwg.add(dwg.polyline(svg_data, stroke='black', fill='none', stroke_width=1))
            
            start_x = float(points[0][0]) * SCALE
            start_y = float(points[0][1]) * SCALE
            f.write(f"G0 X{start_x:.3f} Y{-start_y:.3f}\n")
            f.write(f"G1 Z{Z_DRAW} F{FEED_RATE}\n")
            
            for i in range(1, len(points)):
                px = float(points[i][0]) * SCALE
                py = float(points[i][1]) * SCALE
                f.write(f"G1 X{px:.3f} Y{-py:.3f}\n")
            
            f.write(f"G0 Z{Z_SAFE}\n")
            count += 1
            
        f.write("G0 X0 Y0\nM2\n")
    dwg.save()
    print(f"✅ 저장 완료! 총 {count}개의 획")

# =========================================================
# 실행
# =========================================================
if __name__ == "__main__":
    input_filename = input("파일 명을 입력하세요 (예: face.jpg) : ")
    img_processed = load_and_remove_bg(input_filename)
    
    if img_processed is not None:
        # 하이브리드 함수 호출
        contours = extract_hybrid_contours(img_processed)
        if contours:
            h, w = img_processed.shape[:2]
            save_gcode_and_svg(contours, w, h)

파일 명을 입력하세요 (예: face.jpg) :  input.png


🔄 [1단계] 배경 제거 시작: input.png
✅ 배경 제거 완료
🔄 [2단계] 하이브리드 처리 중 (AI 얼굴 + Canny 디테일)...
✅ 추출 완료: 총 179개의 획 (AI + 디테일)
🔄 [3단계] 파일 생성 시작 (face_drawing_detailed.nc)...
✅ 저장 완료! 총 179개의 획
